# Uso de um prompt que inclui contexto no Bedrock com LangChain

> *Este caderno deve funcionar bem com o kernel **`Data Science 3.0`** no SageMaker Studio*

## Introdução

Neste caderno, mostraremos a você como gerar um e-mail de resposta a um cliente insatisfeito com a qualidade do atendimento ao cliente que recebeu do engenheiro de suporte ao cliente. Para fornecer mais contexto ao modelo, disponibilizaremos o conteúdo do e-mail real enviado pelo cliente insatisfeito.

Por causa do contexto adicional no prompt, o texto gerado pelo grande modelo de linguagem Amazon Titan neste caderno tem uma qualidade muito melhor e maior relevância do que o conteúdo produzido anteriormente por meio de prompts zero-shot.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) é um framework para desenvolvimento de aplicações alimentadas por modelos de linguagem. Os principais aspectos deste framework nos permitem aumentar os grandes modelos de linguagem encadeando vários componentes para criar casos de uso avançados.

Neste caderno, nós usaremos a API do Bedrock fornecida pelo LangChain. O prompt usado neste exemplo cria um modelo personalizado de prompt do LangChain para adicionar contexto à solicitação de geração de texto. 

**Observação:** *este caderno pode ser executado dentro ou fora do ambiente AWS.*

#### Contexto
No exemplo anterior, `01_zero_shot_generation.ipynb`, vimos como usar o framework LangChain para se comunicar com a API do Amazon Bedrock. Neste caderno, tentaremos adicionar um pouco mais de complexidade com a ajuda de `PromptTemplates` para aproveitar o framework LangChain para um caso de uso similar. `PromptTemplates` permite que você crie shells genéricos que podem ser populados com informações posteriormente e obter resultados baseados em diferentes cenários.

Como parte deste caderno, veremos o uso da integração do Amazon Bedrock com o framework LangChain e como ele pode ser usado para gerar texto com a ajuda de `PromptTemplate`.

#### Padrão
Simplesmente ofereceremos a implementação do LangChain da API do Amazon Bedrock com um comando que consiste em uma tarefa, uma instrução e um comando para que o modelo interno gere um resultado sem fornecer exemplos adicionais. Nosso objetivo é demonstrar como poderosos LLMs entendem facilmente a tarefa dada e geram resultados interessantes.

![](./images/bedrock_langchain.jpg)

#### Caso de uso
Para demonstrar a capacidade de geração de modelos no Amazon Bedrock, vamos analisar o caso de uso de geração de um e-mail.

#### Persona
Você é Bob, um gerente de atendimento ao cliente na AnyCompany e alguns de seus clientes não estão satisfeitos com o atendimento ao cliente e estão fornecendo feedbacks negativos sobre o serviço prestado pelos engenheiros de atendimento ao cliente. Você gostaria de pedir suas sinceras desculpas a esses clientes pelo serviço ruim e recuperar a confiança deles. Você precisa da ajuda de um LLM para gerar um lote de e-mails amigáveis e personalizados ao sentimento do cliente da correspondência de e-mail anterior.

#### Implementação
Para atender este caso de uso, mostraremos como gerar um e-mail de agradecimento com base no e-mail anterior do cliente. Usaremos o grande modelo Amazon Titan Text com a integração do LangChain do Amazon Bedrock. 


In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Invocar o modelo de LLM do Bedrock

Começaremos criando uma instância da classe do Bedrock a partir dos LLMs. Para tal, espera-se um `model_id`, que é o ARN do modelo disponível no Amazon Bedrock. 

Uma opção é transmitir um cliente Boto3 criado anteriormente, bem como algum `model_kwarfs`, que pode conter parâmetros como `temperatura`, `topP`, `maxTokenCount` ou `stopSequences` (mais informações sobre parâmetros podem ser encontradas no console do Amazon Bedrock).

Confira a [documentação](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) para obter os IDs de modelo de geração de texto disponíveis no Amazon Bedrock.

Observe que diferentes modelos oferecem suporte a diferentes `model_kwargs`.

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## Criar um modelo de prompt personalizado do LangChain

Ao criar um modelo para o prompt, podemos transmitir diferentes variáveis de comando para ele em cada execução. Isso é útil quando você precisa gerar conteúdo com diferentes variáveis de comando que possa vir a obter de um banco de dados.

Nós codificamos o prompt anteriormente, pode ser que você tenha vários clientes enviando feedbacks negativos semelhantes e agora queira usar cada um dos e-mails desses clientes e respondê-los com um pedido de desculpas, mas também deseje manter a resposta um pouco personalizada. Na célula a seguir, vemos como você pode criar um `PromptTemplate` para alcançar este padrão.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

## Invocar novamente

Invoque usando o modelo do prompt e espere ver uma resposta com curadoria como retorno

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(email)

## Resumo

Para concluir, aprendemos que invocar o LLM sem nenhum contexto pode não trazer os resultados desejados. Ao adicionar contexto e usar o modelo do prompt para restringir o resultado do LLM, conseguimos obter o resultado desejado.